In [ ]:
Vgvet = [-8:0.01:8]*10^-3;
Vvet = [-3:0.01:3]*10^-3;
eps = 10^-6;
CL=653; # e/V
VL=V/2; # V
CR=653; # e/V
VR=-V/2;# V
Cg=145; # e/V

RL=10; # h/2e^2
RR=10; # h/2e^2
T = 4.5*(0.0258/300); #temperature in kelvin transformed in eV
Ec = 1/(2*(CL+CR+Cg)); # eV


for k =1:length(Vgvet);
    Vg=Vgvet(k);# V
for j = 1:length(Vvet);
    V = Vvet(j); # V
#minimum probability accepted as different from 0

#equilibrium charge on the island
#(approximated to the clsest integer, well, charge is quantized)
n0 = CL*VL+CR*VR+Cg*Vg;
#I reset to 0 the vectors I have (eventually) used in the previous loops;
GammaLin = 0;
GammaLout = 0;
Ppos = 0;
Ptemp=1;
#evaluation of the probabilities for P(n) for n>n0 of occupation of the
#island
i = 1;
while Ptemp>eps
#N.B. for i=1 n = floor(n0)!!!
#energy to pay to pass from n to n+1 electrons in the island entering
#from the Left
DeltaE_L = 2*Ec*(floor(n0)+i-1-n0+1/2)+VL;
#from the Right
DeltaE_R = 2*Ec*(floor(n0)+i-1-n0+1/2)+VR;
#
#Rate to go from n to n+1 electrons in the island entering
#from the Left
GammaLin(i)= 1/RL*DeltaE_L/(exp(DeltaE_L/T)-1);
#from the right
GammaRin=1/RR*DeltaE_R/(exp(DeltaE_R/T)-1);
#Rate to go from n+1 to n electrons in the island leaving
#to the Left
GammaLout(i)=1/RL*(-DeltaE_L)/(exp(-DeltaE_L/T)-1);
#to the Right
GammaRout=1/RR*(-DeltaE_R)/(exp(-DeltaE_R/T)-1);
#Total rates in and out
Gammain = GammaLin(i)+GammaRin;
Gammaout = GammaLout(i) + GammaRout;
#Probability P(n+1);
Ptemp = Ptemp*Gammain/Gammaout;
Ppos(i) = Ptemp;
i = i+1;
end
#contribution to the current of the states with n>n0
Ipos = -[1,Ppos(1:end-1)]*GammaLin’ + Ppos*GammaLout’;
#evaluation of the probabilities for P(n) for n<n0 of occupation of the
#island
#I reset to 0 the vectors I have used in the previous while loop;
GammaLin = 0;
GammaLout = 0;
Pneg = 0;
Ptemp = 1;
i = 1;
while Ptemp>eps
#N.B. for i=1 n = floor(n0)!!!
#energy to "pay" (it can be negative and you gain!)
#to pass from n-1 to n electrons in the island entering
#from the Left
DeltaE_L = 2*Ec*(floor(n0)-i-n0+1/2)+VL;
#from the Right
DeltaE_R = 2*Ec*(floor(n0)-i-n0+1/2)+VR;
#
#Rate to go from n-1 to n electrons in the island entering
#from the Left
GammaLin(i)= 1/RL*DeltaE_L/(exp(DeltaE_L/T)-1); #2eV/h
#from the right
GammaRin=1/RR*DeltaE_R/(exp(DeltaE_R/T)-1);
#Rate to go from n to n-1 electrons in the island leaving
#to the Left
GammaLout(i)=1/RL*(-DeltaE_L)/(exp(-DeltaE_L/T)-1);
#to the Right
GammaRout=1/RR*(-DeltaE_R)/(exp(-DeltaE_R/T)-1);
#Total rates in and out
Gammain = GammaLin(i)+GammaRin;
Gammaout = GammaLout(i) + GammaRout;
#Probability P(n+1);
Ptemp = Ptemp*Gammaout/Gammain;
Pneg(i) = Ptemp;
i = i+1;
end
#contribution to the current of the states with n<n0
Ineg = -Pneg*GammaLin’+[1,Pneg(1:end-1)]*GammaLout’;
# construction of the probability distribution P(n)
P = [fliplr(Pneg),1,Ppos];
#definition of the unit conversion for the current. it is in the end
#expressed in Ampere. Notice also the normalization /sum(P)
Unitconv = 1.602*10^-19/(pi*0.6582*10^-15);
IL(j,k) = (Ipos+Ineg)/sum(P)*Unitconv;
P = P/sum(P);
end
end
#calculation of the differential conductance
G = diff(IL,1,1)/diff(Vvet(1:2));
